In [1]:
# %cd '/Users/Adam/Dropbox/PainNarrativesLab/TwitterMining'
# %bookmark twittermining
%cd twittermining

/Users/adam/Dropbox/PainNarrativesLab/TwitterMining
(bookmark:twittermining) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterMining
/Users/adam/Dropbox/PainNarrativesLab/TwitterMining


In [2]:
import os
import date
from CouchDBTools import *
import TwitterSQLService
from TweetDataProcessors import *
from TwitterSearchTools import *
from ObserverAndSubscribers import *
from Loggers import *
from MySQLTools import *

import RedisTools
import CouchDBTools
import ObserverAndSubscribers
import DatabaseAccessObjects.CouchDBDAOs
import TweetDataProcessors
from DatabaseAccessObjects import SqlCredentials

import nltk
from nltk import word_tokenize

from pandas import DataFrame, Series

UPATH = os.getenv("HOME")
BASE = UPATH + '/Dropbox/PainNarrativesLab'
DATAFOLDER = '%s/Data' % BASE
#SQL_CREDENTIALS = "%s/private_credentials/sql_local_credentials.xml" % BASE
SQL_CREDENTIALS = "%s/private_credentials/sql_csun_write_credentials.xml" % BASE
MINER_CREDENTIALS = "%s/private_credentials/sql_miner_credentials.xml" % BASE
LOGFILE = '%s/Desktop/twitter_miner_log.txt' % UPATH

In [3]:

import twitter

from twitter.oauth import read_token_file

def login(credentials=None):
    APP_NAME = 'adaminshanghai_app'
    CONSUMER_KEY = 'W3lp33ZnlUF6gba1y0FLRQ'
    CONSUMER_SECRET = 'vupVVZ81IEnxDpcm7rcJRjRdOIMRqxpQqdhAKvD20'
    TOKEN_FILE = 'out/twitter.oauth'
    ACCESS_TOKEN = '14918616-eq170y0q8pGUbGPevCZTlOiukx4W4xFYejl6yz74'
    ACCESS_TOKEN_SECRET = '9kvz4zul9fNDW831PQawO1NrpcfanDUAqv7Dz2IHw'
    return twitter.Twitter(domain='api.twitter.com', api_version='1.1',
                           auth=twitter.oauth.OAuth(ACCESS_TOKEN, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET))

In [3]:
import tweepy
APP_NAME = 'adaminshanghai_app'
CONSUMER_KEY = 'W3lp33ZnlUF6gba1y0FLRQ'
CONSUMER_SECRET = 'vupVVZ81IEnxDpcm7rcJRjRdOIMRqxpQqdhAKvD20'
TOKEN_FILE = 'out/twitter.oauth'
ACCESS_TOKEN = '14918616-eq170y0q8pGUbGPevCZTlOiukx4W4xFYejl6yz74'
ACCESS_TOKEN_SECRET = '9kvz4zul9fNDW831PQawO1NrpcfanDUAqv7Dz2IHw'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
# Redirect user to Twitter to authorize
#redirect_user(auth.get_authorization_url())

auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

In [9]:
#Get user
r = api.get_user(3097897493)

In [11]:
r2 = api.user_timeline(id=3097897493)

In [14]:
r2

[Status(contributors=None, truncated=False, text=u"My night tonight..strong Painkillers. Minion pj's Netflix and few comforts #endosucks http://t.co/h5QWkIbSab", in_reply_to_status_id=None, id=580075000683724800, favorite_count=0, _api=<tweepy.api.API object at 0x10c070f90>, author=User(follow_request_sent=False, profile_use_background_image=True, _json={u'follow_request_sent': False, u'profile_use_background_image': True, u'profile_text_color': u'333333', u'default_profile_image': False, u'id': 3097897493, u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme1/bg.png', u'verified': False, u'profile_location': None, u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/578691022630309888/q1098UuJ_normal.jpeg', u'profile_sidebar_fill_color': u'DDEEF6', u'entities': {u'description': {u'urls': []}}, u'followers_count': 17, u'profile_sidebar_border_color': u'C0DEED', u'id_str': u'3097897493', u'profile_background_color': u'C0DEED', u'listed_count':

In [ ]:

def get_user_tweets(userid, api):
    tweets = []
    for tweets in tweepy.Cursor(api.user_timeline, id=userid, per_page=20).items():

In [13]:
len(r2)

18

In [5]:
search_terms = ['Spoonie',
                 'CRPS',
                 'Migraine',
                 'RSD',
                 'Fibro',
                 'Fibromyalgia',
                 'Vulvodynia',
                 'ChronicPain',
                 'pain',
                 'endometriosis',
                 'neuropathy',
                 'arthritis',
                 'neuralgia']

In [16]:
import base64
base64.b64decode("Q1NVTlByb3Zvc3Q6MWRldmVsb3BzNQ==")

'CSUNProvost:1develops5'

In [10]:
def search_for_users(api, search_term, filepath=DATAFOLDER):
    """
    Searches for users
    Args:
        api: Tweepy.Api instance
        search_term: String to search for
    """
    #assert(instanceof(api, tweepy.API))
    assert(type(search_term) is str)
    users = []
    processed = 0
    kept = 0
    try:
        for user in tweepy.Cursor(api.search_users, q=search_term, per_page=20).items():
            processed += 1
            if search_term in word_tokenize(user.description):
                users.append(user)
                kept += 1
    except Exception as e:
        print e
    user_dicts = []
    for u in users:
        d = {'userid': u.id_str, 'condition': search_term, 'username': u.name, 'screen_name': u.screen_name, 'description': u.description}
        user_dicts.append(d)
    user_frame = DataFrame(user_dicts)
    today = date.isoformat(date.today())
    filename = "%s/%s_user_search_%s.xlsx" % (filepath, search_term, today)
    user_frame.to_excel(filename)
    print "for %s: processed %s kept %s \n" % (search_term, processed, kept)

In [11]:
search_for_users(api, 'fibro')

[{u'message': u'Invalid value 52 for parameter page parameter is invalid.', u'code': 44}]
for fibro: processed 1040 kept 204 



In [ ]:
search_for_users(api, 'crps')

In [ ]:
search_for_users(api, 'arthritis')

In [ ]:
search_for_users(api, 'vulvodynia')

In [ ]:
search_for_users(api, 'spoonie')

In [ ]:
search_for_users(api, 'neuralgia')

In [19]:
u = users[0]
u.description

u'At http://t.co/8Ahp8mqd4n, we empower patients and caregivers by providing a platform to learn, educate & connect with peers & healthcare professionals'

In [9]:
result = api.search_users(q='migraine', per_page=20, page=1)

In [11]:
result

User(follow_request_sent=False, profile_use_background_image=True, _json={u'follow_request_sent': False, u'profile_use_background_image': True, u'profile_text_color': u'333333', u'default_profile_image': False, u'id': 158418189, u'profile_background_image_url_https': u'https://pbs.twimg.com/profile_background_images/838543705/7c3a4168d44ccbd5ee28b1df980a69b9.png', u'verified': True, u'profile_location': None, u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/378800000867026515/2Kx8Z-Of_normal.png', u'profile_sidebar_fill_color': u'CFD7E4', u'entities': {u'url': {u'urls': [{u'url': u'http://t.co/n08QKGh3d7', u'indices': [0, 22], u'expanded_url': u'http://migraine.com', u'display_url': u'migraine.com'}]}, u'description': {u'urls': [{u'url': u'http://t.co/8Ahp8mqd4n', u'indices': [3, 25], u'expanded_url': u'http://Migraine.com', u'display_url': u'Migraine.com'}]}}, u'followers_count': 13119, u'profile_sidebar_border_color': u'FFFFFF', u'id_str': u'158418189', u'profile_ba

In [7]:
twitter_conn = login()
#r = twitter_conn.search.users(q='migraine', count=100)  # tweets before ealiest record

In [ ]:
twitter_conn.search.users()

In [2]:
twitter.Twitter??

In [1]:
import pandas as pd

In [3]:
d = pd.read_excel('/Users/ars62917/Dropbox/freq_wordlist.xlsx')

In [38]:
import pycurl
from StringIO import StringIO
import json

#print body
for r in body['rows']:
    print r['id']

400304160194568192
400303671516614656
400302725847457792
400302446251347968
400302426281873409
400300849957974016
400300443102097408
400300183743131648
400299882281697280
400295706306822144
400295517390794752
400293886985519104
400292509437267969
400288922007502848
400288835591892993
400286315406962688
400285127743647744
400284364900007937
400283369730088961
400283357634134016
400283314617335809
400282931106574338
400282587371147264
400281884833230848
400281516325888000
400281312482705408
400281080936161280
400280629960384515
400278452164509696
400276469701550080
400275644485222400
400274941704036352
400274802214469632
400274638133288961
400274636887560192
400273747354984448
400273092624519168
400272928887279616
400272388341760001
400271990948261889
400271791198724096
400271102192414720
400271043312365568
400270800680677376
400270755255955456
400270690726604800
400270564897456128
400270535533150208
400270401584254977
400270184654860288
400269038141853696
400262180517724160
400259639159

In [36]:
len(body['rows'])

55

In [43]:
import pycurl
from StringIO import StringIO
import json
import DatabaseAccessObjects.RedisDAOs
class CouchIdGetter(DatabaseAccessObjects.RedisDAOs.RedisDAO):
    """
    This pulls all doc ids from couch to find ones missing from tweetids in redis
    Attributes:
        redisdao: RedisDao object
        couchdao: CouchDBDao object
        starting_tweet: The minimum value stored in mysql
        ending_tweet: The maximum value stored in mysql
        increment: The amount to increase the search ids
        recorded: Counter for number of tweets recorded
    """
    def __init__(self, dbname='n-search-arthritis', db=0):
        self.storage_set_name = 'couchdbtweetIDs'
        if db is not 0:
            DatabaseAccessObjects.RedisDAOs.RedisDAO.__init__(self, db)
        else:
            DatabaseAccessObjects.RedisDAOs.RedisDAO.__init__(self)
        self.minerip = '169.254.203.246:5984'
        self.dbname = dbname
        self.starting_tweet = 4021982852191600065
        self.bottom_limit = 3300000000000000000
        self.increment = 100000000
        self.minid = None
        self.maxid = None
        self.recorded = 0
        
    def check_for_less_than_starting(self):
        """
        Checks to make sure no tweets in db with ids before starting value
        """
        self.maxid = self.starting_tweet
        self.minid = self.maxid - self.increment
        while self.maxid <= self.bottom_limit:
            query = 'GET //_all_docs?startkey="doc2"&limit=2&descending=true HTTP/1.0'
            result = _search(self.minid, self.maxid)
            for row in result:
                if row['id'] is not '_design/index':
                    self.db.sadd(self.storage_set_name, row['id'])
                    self.recorded += 1
            self.maxid = self.minid
            self.minid -= self.increment
        print "%s tweets recorded" % self.recorded
   
    
    def check_for_greater_than_starting(self):
        """
        Checks for values greater than starting
        """
        self.minid = self.starting_tweet
        self.maxid = self.minid + self.increment
        while self.minid <= self.ending_tweet:
            query = 'GET //_all_docs?startkey="doc2"&limit=2&descending=true HTTP/1.0'
            result = _search(self.minid, self.maxid)
            for row in result:
                if row['id'] is not '_design/index':
                    self.db.sadd(self.storage_set_name, row['id'])
                    self.recorded += 1
            self.minid = self.maxid
            self.maxid += self.increment
        print "%s tweets recorded" % self.recorded
            
    def _search(self, startid, stopid):
        """
        Returns:
            List of dicts with key 'id'
        startid = 4021982852191600065
        stopid = 400254646977298433
        """
        buffer = StringIO()
        request = 'http://%s/%s/_all_docs?startkey_docid=%s&endkey_docid=%s&descending=true' % (self.minerip, self.dbname, startid, stopid)
        c = pycurl.Curl()
        c.setopt(pycurl.URL, request)
        c.setopt(c.WRITEDATA, buffer)
        c.perform()
        c.close()
        body = json.loads(buffer.getvalue())
        return body['rows']


In [ ]:
logger = SearchLogger(LOGFILE)

In [ ]:
logger.run_start(10)

In [4]:
cdao = CouchDAO()
cdao.connect('compiled')
rs = RedisTools.RedisService()
cs = CouchDBTools.CouchService()
cs.set_dataservice(cdao)
couch_saver = ObserverAndSubscribers.CouchSaver()
couch_saver.set_dao(cs)
redis_saver = ObserverAndSubscribers.RedisSaver()
redis_saver.set_dao(rs)

In [4]:
minerip = '169.254.203.246'
tweet_id_getter = RedisTools.TweetIdGetter(host=minerip)

In [6]:
tweet_id_getter.count_in_source()

1274099

In [5]:
server = "http://%s:%s" % (minerip, 5984)
print server
tweet_retriever = CouchDBTools.TweetRetriever(server=server)
tweet_retriever._check_connected()

http://169.254.203.246:5984


True

In [6]:
tweet = tweet_retriever.get_tweet('574653811685457920')

ResourceNotFound: (u'not_found', u'missing')

In [ ]:
tr = TweetRetriever()
tweet = tr.get_tweet('574653811685457920')


In [ ]:
d = cdao.db['574653811685457920']
t = tweet_factory.make_tweet(d)
t.hashtags

In [ ]:
tweet = Tweet()
tweet.set_raw_tweet({'id_str': '10001', 'raw_tweet': 'Raw tweet object'})
testtweets = [{'id_str': '10001', 'raw_tweet': 'weet tweet'}, 
              {'id_str': '10002',  'raw_tweet': 'weet tweet'}, 
              {'id_str': '10003', 'raw_tweet': 'weet tweet'}]

In [ ]:
import DatabaseAccessObjects
class TweetIdGetter(DatabaseAccessObjects.RedisDAOs.RedisDAO):
    """
    Attributes:
        temp_queue_name: String name of the redis set to create
        storage_set_name: String name of the set where tweet ids are stored
    """
    def __init__(self, db=0):
        self.temp_queue_name = 'tweetqueue'
        self.storage_set_name = 'tweetIDs'
        if db is not 0:
            DatabaseAccessObjects.RedisDAOs.RedisDAO.__init__(self, db)
        else:
            DatabaseAccessObjects.RedisDAOs.RedisDAO.__init__(self)

    def make_queue(self):
        """
        Copies tweet id set into tweetqueue
        Returns:
            Number of tweetids copied
        """
        #rs = RedisTools.RedisService()
        numtweets = self.db.sunionstore(self.temp_queue_name, self.storage_set_name)
        return numtweets

    def get_tweetid(self):
        """
        Pops a random tweet id off the queue and returns it
        Returns:
            String tweetid (e.g., '574310279153999872')
        """
        return self.db.spop(self.temp_queue_name)
    
    def count_in_source(self):
        """
        Returns the number of tweetids in the source set
        """
        return self.db.scard(self.storage_set_name)
    
    def count_in_queue(self):
        """
        Returns the number of tweetids in the queue
        """
        return self.db.scard(self.temp_queue_name)
    
    def erase_queue(self):
        """
        Removes all values from the temporary queue
        """
        for x in self.db.smembers(self.temp_queue_name): 
            self.db.srem(self.temp_queue_name, x)

class TweetRetriever(CouchDAO):
    def __init__(self, db_name='compiled'):
        CouchDAO.__init__(self)
        self.connect(db_name)
        self.tweet_factory = TweetDataProcessors.TweetFactory()
    
    def get_tweet(self, tweetID):
        """
        Queries couchdb for the tweetid and returns tweet object
        Returns:
            TweetDataProcessors.Tweet object with values all set
        """
        tweetraw = self.db[tweetID]
        tweet = self.tweet_factory.make_tweet(tweetraw)
        return tweet  

In [3]:
cdb = CouchDAO()
cdb.connect('compiled')
rdao = RedisDAO()
for tid in cdb.db:
    rdao.sadd('alltweets', tid)

error: [Errno 61] Connection refused

In [ ]:
couch_saver.save([tweet])

In [ ]:
redis_saver.save(testtweets)

In [ ]:
rs.save_tweet_id('10001')

In [ ]:
rs.loadkeys()

In [ ]:
rs.keys

In [ ]:
#max id
rs.db.sort('tweetIDs', start=0, num=1, desc=True)

In [ ]:
import RedisTools
import CouchDBTools

def checker(tweetid):
    if tweetid is None:
        return False
    else:
        return True

tig = RedisTools.TweetIdGetter()
tweet_retriever = CouchDBTools.TweetRetriever()
numtweets = tig.make_queue()
print "%s tweetids to process" % numtweets
go = True
while True:
    tweetid = tig.get_tweetid()
    go = checker(tweetid)
    print tweetid
    tweet = tweet_retriever.get_tweet(tweetid)
    print tweet.tweetID
        
        

In [ ]:
tig.count_in_source()

In [ ]:
tig = TweetIdGetter()
tig.erase_queue()
tig.count_in_queue()

In [ ]:
tid = tig.get_tweetid()
if tid is None:
    print 'cat'

In [ ]:
rs.db.smembers('tweetqueue')

In [4]:
data = [8, 4]
assert(type(data) is list)

In [ ]:
%run Automine4
getNewerTweets()

In [ ]:
len(Searcher._observers[0]._tweet_queue)

'Spoonie',
'CRPS',
'Migraine',
'RSD',
'Fibro',
'Fibromyalgia',
'Vulvodynia',
'ChronicPain',
'pain',
'endometriosis',
'neuropathy',
'arthritis',
'neuralgia']
Shingles, MS, low back pain

In [ ]:

LOGFILE = '%s/Desktop/twitter_miner_log.txt' % UPATH

DB_NAME = 'n-search-crps'

tag_helper = TagHelpers()
# Initialize mysql handler classes
credentials = SqlCredentials.Credentials()
credentials.load_credentials(SQL_CREDENTIALS)
mysql_dao = TwitterSQLService.SQLService(credentials)
# Initialize services for mysql
TweetService = TweetService()
TweetService.set_dao(mysql_dao)
TweetService.set_helper(tag_helper)

HashtagService = HashtagService()
HashtagService.set_helper(tag_helper)
HashtagService.set_dao(mysql_dao)

extractor = Extractors()

UserService = UserService()
UserService.set_dao(mysql_dao)

Logger = SearchLogger()
Logger.set_log_file(LOGFILE)
Logger.write('ready')

In [12]:
#testing mysql connection
import MySQLdb as msq
import MySQLdb.cursors
from DatabaseAccessObjects import SqlCredentials
SQL_CREDENTIALS = "%s/private_credentials/sql_miner_credentials.xml" % BASE
credentials = SqlCredentials.Credentials()
credentials.load_credentials(SQL_CREDENTIALS)

db = msq.connect(host=credentials.host(), port=credentials.port(),
                                  user=credentials.username(), 
                                  passwd=credentials.password(),
                                  db=credentials.database())#,
                                  # charset='utf8',
                                  # use_unicode=1)
                                 # cursorclass=MySQLdb.cursors.DictCursor)

OperationalError: (2003, "Can't connect to MySQL server on 'adams-imac.csun.edu' (60)")

In [9]:
import MySQLdb as msq

In [ ]:
# Initialize couchdb handler classes
couch_dao = CouchDAO(server='http://169.254.113.17:5984')
couch_dao.connect(DB_NAME)
result = couch_dao.query("""function(doc){emit (doc.id, doc);}""")

In [ ]:
attempts = 0
errors = 0
tweetids = 0

try:
    for r in result:
        attempts += 1
        UserService.recordUser(r['value']['user'])
        TweetService.recordTweetData(r['id'], r['value'])
        tags = tag_helper.getHashtags(r['value'])
     #   tags = Extractors.extractHashtags(r)       
#        for t in r['value']['entities']['hashtags']:
#            tags.append(t['text'])
        if len(tags) > 0:
            #for t in tags:
             #   print t
               # HashtagService._recordTagText(t)
            HashtagService.recordHashtags(r['id'], tags)
        tweetids += 1
except Exception as e:
    errors += 1
    print e

finally:
    print '%i tweets processed' % attempts
    print '%i errors in processing tweets' % errors
    print '%i tweets successfully processed' % tweetids

In [ ]:
result.total_rows

In [ ]:
from BeautifulSoup import BeautifulStoneSoup
def ff(text):
    clean = unicode(BeautifulStoneSoup(text, convertEntities=BeautifulStoneSoup.ALL_ENTITIES))
    return clean


In [ ]:
t = ff('複合性局所疼痛症候群')
t.lower()

In [ ]:
k = []
for j in result:
    k.append(j)
q = k[0]['value']['entities']['hashtags']

In [ ]:
q

In [ ]:
tag_helper.format(q[0]['text'])

In [ ]:
for r in result:
    UserService.recordUser(r['value']['user'])
    TweetService.recordTweetData(r['id'], r['value'])
    for t in r['value']['entities']['hashtags']:
        tags = []
        tags.append(t['text'])
    HashtagService.recordHashtags(r['id'], tags)

In [ ]:
for r in result:
    UserService.recordUser(r['value']['user'])

In [ ]:
for r in result:
    TweetService.recordTweetData(r['id'], r['value'])

In [ ]:
for r in result:
    tags = []
    for t in r['value']['entities']['hashtags']:
        tags.append(t['text'])
    print tags, r['id']
    #HashtagService.recordHashtags(r['id'], tags)

In [ ]:
tweets = []
for r in result:
    tweets.append((r['id'], r['value']['user'], r['value'], r.value['entities']['hashtags']))

In [ ]:
len(result)

In [ ]:
tweets[0]

In [ ]:
(tweetid, tweet, tags, user)

In [ ]:
CREDENTIALS_FILE = "%s/private_credentials/twittercredentials.xml" % BASE
DB_CREDENTIALS = "%s/private_credentials/sql_local_credentials.xml" % BASE
#DB_CREDENTIALS = "/Users/ars62917/Dropbox/PainNarrativesLab/private_credentials/sql_csun_read_credentials.xml"
#DB_CREDENTIALS = "/Users/ars62917/Dropbox/PainNarrativesLab/private_credentials/sql_csun_write_credentials.xml"

In [ ]:
import TwitterLogin

In [ ]:
%debug 
t = TwitterLogin.Login(CREDENTIALS_FILE)

In [ ]:
t.AP

In [ ]:
t.ACCESS_TOKEN

In [ ]:
t.search.tweets(q='cat')

In [ ]:
j.text

In [ ]:
from DatabaseAccessObjects import SqlCredentials
c = SqlCredentials.Credentials()
#c = Credentials()
c.load_credentials(DB_CREDENTIALS)

In [ ]:
c.sql_alchemy_engine()

In [ ]:
class Credentials(object):
   
    def __init__(self):
        self.db_host = "mysqlent.csun.edu"
        self.db_user = ''
        self.db_name = ''
        self.db_password = ''
        self.db_port = '42424'
        
    def host(self):
        return self.db_host;
    
    def username(self):
        return self.db_user
    
    def database(self):
        return self.db_name
    
    def port(self):
        return self.db_port

class RemoteTestingCredentials(Credentials):
        db_host = 'testhost'
        
        def __init__(self):
            Credentials.__init__(self) 
            self.username = 'testusername'
            self.db_name = 'testdbname'
            self.password = 'testpassword'
            self.port = 'testport'

class RemoteReadOnlyCredentials(Credentials):
        
        def __init__(self):
            Credentials.__init__(self)
            #self.db_host = "mysqlent.csun.edu"
            self.username = 'ds_ars629'
            self.db_name = 'ars62917'
            self.password = 'select'
            #self.port = '42424'

In [ ]:
rct = RemoteTestingCredentials()
rct.host()

In [ ]:
from TwitterSqlDaos import TwitterSQLDAO as SqlDao
from SaveToMySQL import *
from TweetSaverService import *
from TwitterDataProcessors import *

In [ ]:
#Load test tweets 
%cd tests
from TwitterProjectTestEntities import *
%cd twitterproject

In [ ]:
#Maker for tweet objects
TF = TweetFactory()
TF.set_extractors(Extractors())
#Make a test tweet object
test2 = TF.make_tweet(test_tweet2)

In [ ]:
test2.user

###MySQL interaction

In [ ]:
#MySQL interaction daos
tweetservice = TweetService()
tweetservice.set_dao(SqlDao(test=True))
tweetservice.set_taghelper(TagHelpers())
#User handling
userservice = UserService()
userservice.set_dao(SqlDao(test=True))
#Hashtag handling
tagservice = HashtagService()
tagservice.set_dao(SqlDao(test=True))
tagservice.set_taghelper(TagHelpers())

In [ ]:
#Test recording user in mysql
userservice.recordUser(t.user)

In [ ]:
tweetservice.recordTweetData(10000230, record)

###Tweet harvester

In [ ]:
#%load_ext autoreload
#%autoreload 2
#dreload(Automine4)
%run Automine4.py
#getNewerTweets()

###Saving in couchdb

In [ ]:
#Test saving in couchdb
from TwitterServiceClasses import CouchService
DB_NAME = 'compiled'
CouchService = CouchService(DB_NAME)
CouchSaver = CouchSaver()
CouchSaver.set_dao(CouchService)

In [ ]:
CouchSaver.save([t])

In [ ]:
###For putting in mysql
"""😷💔📺💊"""

record = {"_id": "401352093598253056",
   "_rev": "2-64861d45d1696f848786975706ad3bf9",
   "truncated": 'false',
   "text": "\"I might have vulvodynia\". Reminds me of Charlotte 😷💔📺💊 #satc #sexology #vulvodynia @ Ganzenplein http://t.co/C1E4pgiu2T",
   "in_reply_to_status_id": 'null',
   "id": 401352093598253060,
   "favorite_count": 0,
   "source": "<a href=\"http://instagram.com\" rel=\"nofollow\">Instagram</a>",
   "retweeted": 'false',
  
   "entities": {
       "symbols": [
       ],
       "user_mentions": [
       ],
       "hashtags": [
           {
               "indices": [
                   56,
                   61
               ],
               "text": "satc"
           },
           {
               "indices": [
                   62,
                   71
               ],
               "text": "sexology"
           },
           {
               "indices": [
                   72,
                   83
               ],
               "text": "vulvodynia"
           }
       ],
       "urls": [
           {
               "url": "http://t.co/C1E4pgiu2T",
               "indices": [
                   98,
                   120
               ],
               "expanded_url": "http://instagram.com/p/gvPYnCDEPm/",
               "display_url": "instagram.com/p/gvPYnCDEPm/"
           }
       ]
   },
   "in_reply_to_screen_name": 'null',
   "in_reply_to_user_id": 'null',
   "retweet_count": 0,
   "id_str": "401352093598253056",
   "favorited": 'false',
   "user": {
  
       "id": 319297935,
       "verified": 'false',
         "entities": {
           "url": {
               "urls": [
                   {
                       "url": "http://t.co/845k6L9q1k",
                       "indices": [
                           0,
                           22
                       ],
                       "expanded_url": "http://www.facebook.com/niels.jacobus",
                       "display_url": "facebook.com/niels.jacobus"
                   }
               ]
           },
           "description": {
               "urls": [
               ]
           }
       },
       "followers_count": 69,
   
       "id_str": "319297935",
   
       "utc_offset": 7200,
       "statuses_count": 137,
       "description": "Student seksuologie @KU_Leuven | KSA",
       "friends_count": 81,
       "location": "Leuven",
       "screen_name": "NielsJacobus",
       "lang": "en",
       "profile_background_tile": 'false',
       "favourites_count": 19,
       "name": "Niels ☕",
       "url": "http://t.co/845k6L9q1k",
       "created_at": "Fri Jun 17 21:54:43 +0000 2011",
        "time_zone": "Athens",
   },
   
   "lang": "de",
   "created_at": "Fri Nov 15 14:12:50 +0000 2013",
   
   "place": 'null',
   "metadata": {
       "iso_language_code": "de",
       "result_type": "recent"
   }
                            }

In [ ]:
%cd uscproject
#General tools
import sys
import locale
import json
import time
from random import shuffle
import itertools #For set operations
from urllib2 import URLError

import networkx as nx
#Twitter specific
import twitter
import twitter_text # easy_install twitter-text-py

from TwitterLogin import login
from TwitterUtilities import makeTwitterRequest

#Data storage
import redis
import couchdb
from couchdb.design import ViewDefinition

t = login()

#%%bash couchdb
import TwitterServiceClasses as TSC
import TwitterSearcher as TS
import TwitterDataProcessors as TDP

#Data manipulation and cleaning
from TwitterDataProcessors import Tags
from TwitterDataProcessors import Extractors
from TwitterDataProcessors import TagsFromSearch

In [ ]:
#log setup
LOGFILE = '/Users/adam/Desktop/twitter_miner_log.txt'

import logging

class LogWriter: 
  def write(self, stuff):
    f = open(LOGFILE, 'a')
    f.write(stuff)
    f.close()

def stdin2file(func, file_like):
  def innerfunc(*args, **kwargs):
    old = sys.stdout
    sys.stdout = file_like
    try:
      return func(*args, **kwargs)
    finally:
      sys.stdout = old
  return innerfunc
"""
   # Decorator to redirect any calls to stdout (print) to LogWriter
"""
"""    
def redirect_stdin(fi):
    def stdin2file(func, fi):
        def innerfunc(*args, **kwargs):
          old = sys.stdout
          sys.stdout = fi
          try:
            return func(*args, **kwargs)
          finally:
            sys.stdout = old
        return innerfunc
    return stdin2file
def t(): pass
"""
#@redirect_stdin(t, LogWriter())
def jip():
    print 'jop'
jip()

jip = stdin2file(jip, LogWriter())

jip()


In [ ]:
LogWriter().write('j')

In [ ]:
out.getvalue()

In [ ]:
x = str(45)
x.lower()

In [ ]:
logging.basicConfig(filename=LOGFILE, mode='a')
logging.warning('jip')

#Loading to mysql

In [ ]:
"""
The following was moved to SaveToMySQL.py
"""

#TODO: Better error handling. 
#TODO: Maybe get all ids then transfer one by one
import TwitterSQLService

from BeautifulSoup import BeautifulStoneSoup
class TweetError:
    def __init__(self, tweetID):
        self.tweetID = tweetID
    def __repr__(self):
        return "%s went bad on tweetID %s" % (self.kind, self.tweetID)

class TweetServiceError(TweetError):
    def __init__(self, tweetID):
        self.kind = 'TweetService'
        TweetError.__init__(self, tweetID)
        
class HashtagServiceError(TweetError):
    def __init__(self, tweetID):
        self.kind = 'HashtagService'
        TweetError.__init__(self, tweetID)

class UserServiceError(TweetError):
    def __init__(self, tweetID):
        self.kind = 'UserService'
        TweetError.__init__(self, tweetID)


class TweetService(TwitterSQLService.SQLService):
    def __init__(self, test=False, local=True):
        TwitterSQLService.SQLService.__init__(self, test, local)
         
    def HTMLEntitiesToUnicode(self, text):
        """Converts HTML entities to unicode.  For example '&amp;' becomes '&'."""
        text = unicode(BeautifulStoneSoup(text, convertEntities=BeautifulStoneSoup.ALL_ENTITIES))
        return text

    def recordTweetData(self, tweetID, tweet):
        """
        @param tweetid ID of the tweet from id_str
        """
        ##Make sure urder id is recorded
        try:
            text = tweet['text']
#            text = self.HTMLEntitiesToUnicode(tweet['text'])
            self.query = """INSERT INTO tweets (tweetID, tweetText, favorite_count, source, retweeted, in_reply_to_screen_name, retweet_count, favorited, userID, lang, created_at)
             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
            self.val = [tweetID, text, 
                      tweet['favorite_count'], tweet['source'], 
                      tweet['retweeted'], tweet['in_reply_to_screen_name'], 
                      tweet['retweet_count'], tweet['favorited'], 
                      tweet['user']['id'], tweet['lang'], tweet['created_at']]
            self.executeQuery()
            ##run query
        except:
            raise TweetServiceError(tweetID)
            #print "Error in recording tweet %s. \n %s" % (tweetID, e)

class HashtagService(TwitterSQLService.SQLService):
    def __init__(self, test=False, local=True):
        TwitterSQLService.SQLService.__init__(self, test, local)
    
    def tagFormatter(self, tagText):
        """
        Make the appropriate transformations to the hashtag
        """
        try:
            tag = unicode(BeautifulStoneSoup(tagText, convertEntities=BeautifulStoneSoup.ALL_ENTITIES))
            tag = str(tag)
            return tag.lower()
        except:
            raise HashtagServiceError(tagText)
        finally:
            return tagText
            #print "Tag formatting failed for tag text: %s \n %s" % (tagText, e)

    def getHashtags(self, record):
        hashtags = []
        for r in record['entities']['hashtags']:
            hashtags.append(r['text'])
        return hashtags

    def getTagID(self, tagText):
        """
        The text of the hashtag to get
        """
        tag = self.tagFormatter(tagText)
        self.query = """SELECT tagID FROM hashtags WHERE hashtag = %s"""
        self.val = [tagText]
        self.returnOne()
        return self.results
    
    def recordTagText(self, tagText):
        """
        If the tag is new, records the tag text
        """
        tag = self.tagFormatter(tagText)
        self.query = """INSERT INTO hashtags (hashtag) VALUES (%s)"""
        self.val = [tag]
        self.executeQuery()
    
    def recordTagAssoc(self, tweetID, tagID):
        """
        Associates a tag with a tweet
        """
        self.query = """INSERT IGNORE INTO tweetsXtags (tweetID, tagID) VALUES (%s, %s)"""
        self.val = [tweetID, tagID]
        self.executeQuery()
        
    def recordHashtags(self, tweetID, tags):
        """
        @param tweetid int ID of the tweet from id_str
        @param tags list of tag strings
        """
        for tag in tags:
            try:
                #Lookup tag id
                tagID = self.getTagID(tag)
                if tagID == None:
                    #Insert the tag text into hashtags table
                    self.recordTagText(tag)
                    tagID = self.getTagID(tag)
                if tagID != None: #Now have id. Make association
                   self.recordTagAssoc(tweetID, tagID['tagID']) 
            except:
                raise HashtagServiceError(tweetID)
            #except Exception as e:
            #    print "Error in recording hashtags for tweetid %s. \n Tag: %s \n %s" % (tweetID, tag, e)    

class UserService(TwitterSQLService.SQLService):
    def __init__(self, test=False, local=True):
        TwitterSQLService.SQLService.__init__(self, test, local)
        self.ignorecolumns = ['id', 'entities',
        'follow_request_sent',
        'profile_use_background_image',
        'profile_text_color',
        'profile_image_url_https',
        'profile_sidebar_fill_color',
        'is_translator',
        'geo_enabled',
        'protected',
        'default_profile_image','profile_link_color', 'profile_image_url','notifications',
        'profile_background_image_url_https', 'profile_background_color','profile_banner_url',
        'profile_background_image_url', 'profile_background_tile','contributors_enabled',
        'profile_sidebar_border_color','default_profile', 'following','listed_count',
        'follow_request_sent']
       
    def checkUser(self, userID):
        """
        Used to check whether user already exists
        """
        self.query = """SELECT userID FROM users WHERE userID = %s"""
        self.val = [userID]
        self.returnAll()
        if len(self.results) > 0:
            return True
        else:
            return False
    
    def createUser(self, userDict):
        """
        Creates the user record. Only necessarily does useriD
        @param userDict Dictionary containing user info
        """
        #Insert the userID
        self.query = """INSERT INTO users (userID) VALUES (%s)"""
        self.val = [userDict['id']]
        self.executeQuery()
        if self.checkUser(userDict['id']) == True:
            touse = [x for x in userDict.keys() if x not in self.ignorecolumns]
            #Then update
            for k in touse:
                #try:
                #if (k != 'id') and (k != 'entities'):
                self.query = """UPDATE users SET %s = %%s WHERE userID = %%s""" % k
                self.val = [userDict[k], userDict['id']]
                self.executeQuery()       
                #except Exception as e:
                 #   print e             
        else: 
            raise UserServiceError(userDict['id'])

    def recordUser(self, userDict):
        if self.checkUser(userDict['id']) == True:
            return True
        else:
            self.createUser(userDict)
            if self.checkUser(userDict['id']) == True:
                return True
            else:
                raise UserServiceError(userDict['id'])


In [ ]:
#connect to imac couchdb
%run CouchDBDAOs
C =  CouchDAO('http://192.168.1.111:5984')
C.connect('compiled')

#Run the conversion
TEST = False; LOCAL = True
us = UserService(test=TEST, local=LOCAL)
ts = TweetService(test=TEST, local=LOCAL)
hts = HashtagService(test=TEST, local=LOCAL)
"""
for tweet in C.db.query("""function(doc){emit (doc.id, doc);}"""):
    try:
        tweetID = tweet.key
        record = tweet.value
        #First put user into users table
        us.recordUser(record['user'])
        #Second, record tweet
        ts.recordTweetData(tweetID, record)
        #Third, record hashtags
        hts.recordHashtags(tweetID, hts.getHashtags(record))
    except ValueError:
        print 'value error for id: %s' % tweetID
    except KeyError:
        print 'key error for %s' % tweetID
    except TweetError, x:
        print x
#    except Exception as e:
 #       print 'Error on tweetID: % \n %s' % (tweetID, e)
"""

In [ ]:
tw = C.db.query("""function(doc){emit (doc.id, doc);}""")
tw[1]


In [ ]:
TEST = True; LOCAL = True
us = UserService(test=TEST, local=LOCAL)
ts = TweetService(test=TEST, local=LOCAL)
hts = HashtagService(test=TEST, local=LOCAL)

##Record tweet
tid = record["id_str"]
#First put user into users table
us.recordUser(record['user'])
#Second, record tweet
ts.recordTweetData(tid, record)
#Third, record hashtags
hts.recordHashtags(tid, hts.getHashtags(record))

In [ ]:
record.keys()
##Get tags from record
tags = record['entities']['hashtags']
taglist = []
for t in tags:
    taglist.append(t['text'])

In [ ]:
from CouchDBDAOs import CouchDAO as cdao
C = cdao()

In [ ]:
reload(cdao)

###Create columns of tweets table
ALTER TABLE  tweets 
ADD COLUMN userID BIGINT(30) NOT NULL,
ADD COLUMN tweetText TEXT NOT NULL, 
ADD COLUMN favorite_count INT(10), 
ADD COLUMN source VARCHAR(100), 
ADD COLUMN retweeted VARCHAR(10), 
ADD COLUMN in_reply_to_screen_name VARCHAR(100), 
ADD COLUMN retweet_count INT(10), 
ADD COLUMN favorited VARCHAR (10),  
ADD COLUMN lang VARCHAR(100), 
ADD COLUMN created_at VARCHAR(100),
ADD COLUMN updated TIMESTAMP;
###Create columns of users table
ALTER TABLE users 
ADD COLUMN lang VARCHAR(100),
ADD COLUMN utc_offset VARCHAR(100),
ADD COLUMN verified VARCHAR(100),
ADD COLUMN description VARCHAR(100),
ADD COLUMN friends_count INT(10),
ADD COLUMN url VARCHAR(100),
ADD COLUMN time_zone VARCHAR(100),
ADD COLUMN created_at VARCHAR(100),
ADD COLUMN name VARCHAR(100),
ADD COLUMN entities VARCHAR(100),
ADD COLUMN followers_count VARCHAR(100),
ADD COLUMN screen_name VARCHAR(100),
ADD COLUMN id_str VARCHAR(100),
ADD COLUMN favourites_count INT(10),
ADD COLUMN statuses_count INT(10),
ADD COLUMN id VARCHAR(100),
ADD COLUMN location VARCHAR(100)


In [ ]:
RS = TSC.RedisService()

In [ ]:
import RedisDAOs
class RemoteRedisService(RedisDAOs.RedisDAO):
    def __init__(self, host, port, db=0):
        RedisDAOs.RedisDAO.__init__(self, host, port, db)
    #    self.loadkeys()
    
    def loadkeys(self):
        self.keys = self.db.keys('*')
rrs = RemoteRedisService(host='192.168.1.111', port=6379)
#rrs.db.info()
#rrs.loadkeys()

In [ ]:
keys = RS.db.keys('unique_tweets:tweet_id:*')

In [ ]:
[TSC.RedisFormat.tweet_id_remove(k) for k in keys]
keys[:4]

In [ ]:
mykey = 'tweetIDs'
v = 99
#RS.db.zadd(mykey, name1=23, name2=63, name3=32, name4=44)
RS.db.sadd(mykey, v)
#RS.db.zadd(mykey, name=v)

In [ ]:
RS.db.sort(mykey, start=0, num=1, desc=True)

In [ ]:
#RS.keys.sort()
mx = max(RS.keys)

In [ ]:
mx = max(RS.keys)
TSC.RedisFormat.tweet_id_remove(mx)

In [ ]:
RS.keys[:4]

In [ ]:
import datetime

In [ ]:
from datetime import time
datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y")
#date.isoformat(date.today())


In [ ]:
import networkx as nx
#Save each day's tags in a graph. That way saves processing time; later can combine graphs
today = date.isoformat(date.today())
graphFile = 'tweet_hashtags_graph_%s.gexf' % today
#g = nx.Graph()
try:
    graph = nx.read_gexf(graphFile)
except:
    g = nx.Graph()
    try:
        nx.write_gexf(g, graphFile)
        graph = nx.read_gexf(graphFile)
    except:
        print 'Something is very wrong with the graphfile retrieval'

In [ ]:
nx.info(graph)

### Working on integrating builder

In [ ]:
%cd uscproject
from TwitterDataProcessors import Extractors
from TwitterLogin import login

twitter_conn = login()
search_results = twitter_conn.search.tweets(q='pain', count=100)
#for status in self.search_results['statuses']:

statuses = search_results['statuses']  # This is the list returned which contains all the tweets


In [ ]:
import networkx as nx
import sys
graphFile = 'tweet_hashtags_graph_MASTER.gexf'
try:
    graph = nx.read_gexf(graphFile)
    print 'Start graph info', nx.info(graph)
except Exception as e:
    print 'Error loading graph from %s: %s' % (graphFile, e)    
    #Prevent further execution
    sys.exit()
#Add to graph
try:
    for tweet in statuses:
        tweet_tuples = []
        entities = Extractors.getEntities(tweet)
        if len(entities['hashtags']) >= 1:
            list_of_hashtags = entities['hashtags']
            for tag in list_of_hashtags:
                tag_cleaned = str(tag['text'])
                tag_cleaned = tag_cleaned.lower()
                tweetid = tweet['id_str']
                tweet_tuple = (tweetid, tag_cleaned)
                tweet_tuples.append(tweet_tuple)
            try:
                graph.add_edges_from(tweet_tuples)
            except Exception as e:
                print 'Error adding edges to graph %s', e
except Exception as e:
    print 'Error in building graph %s' % e
finally:
    nx.write_gexf(graph, graphFile)
    print 'Saved graph info : ', nx.info(graph)

In [ ]:
tweet_tuples

In [ ]:
import networkx as nx
graphFile = 'tweet_hashtags_graph_MASTER.gexf'
try:
    graph = nx.read_gexf(graphFile)
except Exception as e:
    print 'Error loading graph from %s: %s' % (graphFile, e)    
#Add to graph
try:
    graph.add_edges_from(tweet_tuples)
except Exception as e:
    print 'Error adding edges to graph %s', e
    
nx.write_gexf(graph, self.graphFile)
        

In [ ]:
['trtags = Extractors.extractHashtags(statuses)

In [ ]:
from TwitterDataProcessors import Extractors
from TwitterLogin import login

twitter_con = login()
search_results = twitter_conn.search.tweets(q=self.query, count=100)
#for status in self.search_results['statuses']:

statuses = self.search_results['statuses']  # This is the list returned which contains all the tweets

for tweet in statuses:
tags = Extractors.extractHashtags(tweet)
#tags = tweet['hashtags']
tweetkey = tweet['id_str']
except Exception as e:
print 'Failed to extract %s' % e
##tags = []
#if tweet.value is not None:
for tag in tags: #tweet.value:
try:
        #Make lowercase and convert to string
        #tag_cleaned = str(tag['text'])
        tag_cleaned = str(tag)
        tag_cleaned = tag_cleaned.lower()
        #Make tuple of tweetid and tag
        self.tweet_tuples.append((tweetkey, tag_cleaned))
        #self.tweet_tuples.append((tweet.key, tag_cleaned))
except:
        self.errors.append(tweetkey)
except Exception as e:
print 'bad thing happened %s' % e
#pass


### ----end builder

#To Do
Try doing the analysis on twitter users rather than tweets

# Move below here

# Mining tweets

In [ ]:
reload(TSC)
#doc = {'key' : 1, 'value' : test}
t = statuses[0]
#t['_id'] = t['id_str']
t['_id'] = str(1)
#TSC.CouchSaver.save('search-pain', t) 
cs = TSC.CouchService('search-pain')
cs.db.save(t)

In [ ]:
#Mine tweets (this is best taken care of with TwitterAutoMine in idle)
reload(TS)
ts = TS.Search()
ts.execute('#spoonie', 'False')
search_terms = ['#Spoonie', '#CRPS', '#Migraine', '#TN', '#RSD', '#Fibro', '#Fibromyalgia']
#for s in search_terms:
    #ts.execute(s, 'False')

In [ ]:
test = {u'contributors': None,
 u'coordinates': None,
 u'created_at': u'Tue May 14 21:13:18 +0000 2013',
 u'entities': {u'hashtags': [],
  u'symbols': [],
  u'urls': [],
  u'user_mentions': []},
 u'favorite_count': 0,
 u'favorited': False,
 u'geo': None,
 u'id': 334416152052768768,
 u'id_str': u'334416152052768768',
 u'in_reply_to_screen_name': None,
 u'in_reply_to_status_id': None,
 u'in_reply_to_status_id_str': None,
 u'in_reply_to_user_id': None,
 u'in_reply_to_user_id_str': None,
 u'lang': u'en',
 u'metadata': {u'iso_language_code': u'en', u'result_type': u'recent'},
 u'place': None,
 u'retweet_count': 0,
 u'retweeted': False,
 u'source': u'<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 u'text': u'For the ones who hide the truth to protect others. For the ones who have the truth when everybody is lying. For those who suffer the truth.',
 u'truncated': False,
 u'user': {u'contributors_enabled': False,
  u'created_at': u'Wed Apr 10 13:57:07 +0000 2013',
  u'default_profile': True,
  u'default_profile_image': False,
  u'description': u'Always forward unto dawn',
  u'entities': {u'description': {u'urls': []}},
  u'favourites_count': 17,
  u'follow_request_sent': None,
  u'followers_count': 24,
  u'following': None,
  u'friends_count': 49,
  u'geo_enabled': False,
  u'id': 1341982466,
  u'id_str': u'1341982466',
  u'is_translator': False,
  u'lang': u'es',
  u'listed_count': 0,
  u'location': u'',
  u'name': u'Juan [CRPS]',
  u'notifications': None,
  u'profile_background_color': u'C0DEED',
  u'profile_background_image_url': u'http://a0.twimg.com/images/themes/theme1/bg.png',
  u'profile_background_image_url_https': u'https://si0.twimg.com/images/themes/theme1/bg.png',
  u'profile_background_tile': False,
  u'profile_image_url': u'http://a0.twimg.com/profile_images/3502376973/5b0cf2c9a0fe53047c8b19d75378d251_normal.jpeg',
  u'profile_image_url_https': u'https://si0.twimg.com/profile_images/3502376973/5b0cf2c9a0fe53047c8b19d75378d251_normal.jpeg',
  u'profile_link_color': u'0084B4',
  u'profile_sidebar_border_color': u'C0DEED',
  u'profile_sidebar_fill_color': u'DDEEF6',
  u'profile_text_color': u'333333',
  u'profile_use_background_image': True,
  u'protected': False,
  u'screen_name': u'Juan_CRPS_',
  u'statuses_count': 87,
  u'time_zone': None,
  u'url': None,
  u'utc_offset': None,
  u'verified': False}}

# Processing tweets

In [ ]:
#also need to store the total number of hashtags; tags per posts; total posts
# Need to write the hashtag isolator into the view --so it won't retrieve every damn record

### Condense tags into Redis

In [ ]:
#check new size of redis db
reload(TSC)
RS = TSC.RedisService()
info = RS.db.info() 
size = info['db0']
print size


In [ ]:
3.35pm 13424

In [ ]:
r = redis.Redis(host='localhost', port=6379, db=0)
r.hgetall(333940296658325505)

#RS.db.getse)['333940296658325505']

In [ ]:
tid = self.search_results['statuses'][0]['id_str']
print tid
present = self.redis_service.lookup_tweet_id(tid)
if present == False:
#Record in redis
self.redis_service.save_tweet_id_to_master_list(tid)
self.tweets += self.search_results['statuses']
else:
print 'already have tweet %s' % str(tid)
except Exception as e:
print 'problem looking up tweet'
j['statuses'][0]['id_str']
					

In [ ]:
results = []
keys = RS.db.keys('*')
keys[:5]
if 'unique_tweets:tweet_id:333219385986670593' in keys:
    print 'jip'
else:
    print 'gland'

In [ ]:
#RS.get_all_tags()
#RS.results[:4]
#db['unique_tweets:tweet_id:333940296658325505']
#RS.db.getset('unique_tweets:tweet_id:333940296658325505')
RS.lookup_tweet_id(333948787221463041)

In [ ]:
RS.remove_redis_format_from_results()
len(RS.clean_results)

In [ ]:
len(pairs)

In [ ]:
reload(TSC)
a = TSC.RedisFormat.tag_add('catfish') 
b = TSC.RedisFormat.tweet_id_add(45)

In [ ]:
TSC.RedisFormat.tweet_id_remove(b)

## Make networkx graph

In [ ]:
 import pickle 
 filehandler = open('pairs', 'r') 
 pair_data = pickle.load(filehandler) 



In [ ]:
import networkx as nx

g = nx.Graph()
g.add_edges_from(pair_data)

In [ ]:
nx.write_gexf(g, 'charts_and_graphs/hashtagtest2.gexf')


#---------------------------------

In [ ]:
r = CS.db.query("""function(doc){emit (doc.entities.hashtags);}""")


In [ ]:
def get_tweet_texts():
    query = db.query("""function(doc){emit (null, doc.text);}""")
    text = []
    for i in query:
        text.append(i.value)
    return text


# ------------------ various book things

In [ ]:
def makeTwitterRequest(t, twitterFunction, max_errors=3, *args, **kwArgs): 
    wait_period = 2
    error_count = 0
    while True:
        try:
            return twitterFunction(*args, **kwArgs)
        except twitter.api.TwitterHTTPError, e:
            error_count = 0
            wait_period = handleTwitterHTTPError(e, t, wait_period)
            if wait_period is None:
                return
        except URLError, e:
            error_count += 1
            print >> sys.stderr, "URLError encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise

def _getRemainingHits(t):
    return t.account.rate_limit_status()['remaining_hits']

# Handle the common HTTPErrors. Return an updated value for wait_period
# if the problem is a 503 error. Block until the rate limit is reset if
# a rate limiting issue
def handleTwitterHTTPError(e, t, wait_period=2):
    if wait_period > 3600: # Seconds
        print >> sys.stderr, 'Too many retries. Quitting.'
        raise e
    if e.e.code == 401:
        print >> sys.stderr, 'Encountered 401 Error (Not Authorized)'
        return None
    elif e.e.code in (502, 503):
        print >> sys.stderr, 'Encountered %i Error. Will retry in %i seconds' % (e.e.code,
                wait_period)
        time.sleep(wait_period)
        wait_period *= 1.5
        return wait_period
    elif _getRemainingHits(t) == 0:
        status = t.account.rate_limit_status()
        now = time.time()  # UTC
        when_rate_limit_resets = status['reset_time_in_seconds']  # UTC
        sleep_time = max(when_rate_limit_resets - now, 5) # Prevent negative numbers
        print >> sys.stderr, 'Rate limit reached: sleeping for %i secs' % (sleep_time, )
        time.sleep(sleep_time)
        return 2
    else:
        raise e



In [ ]:
import sys
import twitter
import couchdb
from couchdb.design import ViewDefinition
from twitter__util import makeTwitterRequest
from twitter__login import login

Q = sys.argv[1]
MAX_PAGES = 5

server = couchdb.Server('http://localhost:5984')
DB = 'search-%s' % (Q.lower().replace('#', '').replace('@', ''), )

t = login()
search_results = t.search.tweets(q=Q, count=100)
tweets = search_results['statuses']

for _ in range(MAX_PAGES-1): # Get more pages
    next_results = search_results['search_metadata']['next_results']

    # Create a dictionary from the query string params
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ]) 

    search_results = t.search.tweets(**kwargs)
    tweets += search_results['statuses']

    if len(search_results['statuses']) == 0:
        break

    print 'Fetched %i tweets so far' % (len(tweets),)

# Store the data
try:
    db = server.create(DB)
except couchdb.http.PreconditionFailed, e:
    # Already exists, so append to it (but be mindful of appending duplicates with repeat searches.)
    # The refresh_url in the search_metadata or streaming API might also be
    # appropriate to use here.
    db = server[DB]

db.update(tweets, all_or_nothing=True)
print 'Done. Stored data to CouchDB - http://localhost:5984/_utils/database.html?%s' % (DB,)

In [ ]:
# Recorded in TwitterServiceClasses
def redis_result_list_fixer(redis_result_list):
    tags = []
    for r in redis_result_list:
        t = tag_breaker(r)
        tags.append(t)
    return tags

def tag_remove_redis_format(redisified_tag):
    junk, sep, tag = redisified_tag.partition(':')
    return tag

# ------------------------ MY ATTEMPT

In [ ]:
%cd uscproject
import twitter__util
reload(twitter__util)

In [ ]:
t.users.show(screen_name='PainPhD')

In [ ]:
max_id_of_search = crps['search_metadata']['max_id'] #Include this in subsequent searches to make sure no overlap

In [ ]:
crps2 = t.search.tweets(q='#CRPS', max_id=max_id_of_search)

In [ ]:
#MY attempt

i = 1
limit = 5
records = []
maxids = ['', '']
mid = None

def tweetget(i, mid):
    if i == 1:
        ht = t.search.tweets(q='#CRPS')
    else:
        ht = t.search.tweets(q='#CRPS', max_id=mid)
    #Store maxid for next loop
    maxid = ht['search_metadata']['max_id']
    mid = maxid
    maxids.append(maxid)
    #Get hashtags
    tags = extractHashtags(ht)
    records.append(tags)
    print i
    print maxids

while i <= limit:
    tweetget(i, mid)
    i += 1

In [ ]:
crps = t.search.tweets(q='#CRPS')
max_id_of_search = crps['search_metadata']['max_id'] #Include this in subsequent searches to make sure no overlap
crps2 = t.search.tweets(q='#CRPS', max_id=max_id_of_search)
tagset = extractHashtags(crps)

In [ ]:
i = 0
limit = 5

maxid = None

while i <= limit:
    if i = 0:
        crps = t.search.tweets(q='#CRPS')
        maxid = crps['search_metadata']['max_id'] #Include this in subsequent searches to make sure no overlap
        return maxid
    
    else:
        crps2 = t.search.tweets(q='#CRPS', max_id=maxid)